In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
import logging

Preprocess data as in https://colab.research.google.com/drive/1tdgzzgPODIzrQ47CG1PKw9Gi6WHrONWf?authuser=1#scrollTo=y0BGiw2ESzV5&line=84&uniqifier=1, and https://colab.research.google.com/drive/1tdgzzgPODIzrQ47CG1PKw9Gi6WHrONWf?authuser=1#scrollTo=D5EhRjxxK8p3&line=1&uniqifier=1.

In [ ]:
import pandas as pd
def merge_cr5(row):
    cr5_cols = ['CR5A', 'CR5B', 'CR5C', 'CR5D', 'CR5E']
    cr5_flags = [1, 2, 3, 4, 5]
    cr5_values = [not pd.isna(row[col]) for col in cr5_cols]  # generates a Boolean list indicating whether each column has a value

    if sum(cr5_values) > 1:  # 7 represents two or more races
        return 7
    elif row['CR4'] in [2, 3, 4, 5] and any(cr5_values):
        return 7
    elif sum(cr5_values) == 1:
        return cr5_flags[cr5_values.index(True)]
    elif row['CR4'] in [2, 3, 4, 5]:
        return 6
    else:
        return None

# Preprocess 2012 data
data_2012 = pd.read_excel('/content/drive/My Drive/Alabama12_ms.xlsx')
data_2012 = data_2012.copy()
data_2012['CR5'] = data_2012.apply(merge_cr5, axis=1)
selected_columns = ['CR2', 'CR3', 'CR5', 'CR7', 'CR11', 'CR12', 'CR13', 'CR21', 'CR22', 'CR23', 'CR54', 'CR56']
data_2012 = data_2012.loc[:, selected_columns]

data_2014 = pd.read_excel('/content/drive/My Drive/Alabama14_ms.xlsx')
data_2014 = data_2014.copy()
data_2014['CR5'] = data_2014.apply(merge_cr5, axis=1)
data_2014 = data_2014.loc[:, selected_columns]

data_2016 = pd.read_csv('/content/drive/My Drive/alabama16_6rigions_ms.sas7bdat.csv')
data_2016 = data_2016.copy()
data_2016['CR5'] = data_2016.apply(merge_cr5, axis=1)
data_2016 = data_2016.loc[:, selected_columns]

# Load the 2024 input data
inputs_2024 = pd.read_csv('/content/drive/My Drive/2024_sample_data_not_survey.csv')
required_columns = ['CR2', 'CR3', 'CR5']
# Ensure the 2024 input data columns match the required columns
conditions_2024 = inputs_2024.loc[:, required_columns]

In [ ]:
data_2012.fillna(0, inplace=True)
data_2014.fillna(0, inplace=True)
data_2016.fillna(0, inplace=True)
conditions_2024.fillna(0, inplace=True)
data_2012.insert(0, 'Year', 2012)
data_2014.insert(0, 'Year', 2014)
data_2016.insert(0, 'Year', 2016)
conditions_2024.insert(0, 'Year', 2024) # including year feature

# Combine the datasets into one DataFrame
combined_data = pd.concat([data_2012, data_2014, data_2016], ignore_index=True)

I decided to use regression model for the time series data. I incorporate feature engineering in this process, so that enriching the dataset with valuable information that helps the model learn patterns more effectively. This is essential for time series data, where past behavior can strongly influence future predictions.


In [ ]:
#Computes the accuracy of predictions by checking if the relative error between predicted and actual values is less than a specified threshold
def calculate_accuracy(y_true, y_pred, threshold=0.1):
    return ((abs(y_true - y_pred) / y_true) < threshold).mean()

# Creates lag features for specified columns, which represent previous time steps
def generate_lag_features(df, features, lags=[1]):
    for lag in lags:
        for feature in features:
            df[f"{feature}_lag{lag}"] = df[feature].shift(lag)
    return df

#Computes moving averages for the specified columns, smoothing out fluctuations in the data
def add_moving_averages(df, features, windows=[3]):
    for window in windows:
        for feature in features:
            df[f"{feature}_ma{window}"] = df[feature].rolling(window=window).mean()
    return df

# Adds difference features, which calculate the change between the current value and the previous value for each feature, helping to identify trends or shifts over time.
def add_diff_features(df, features):
    for feature in features:
        df[f"{feature}_diff"] = df[feature].diff()
    return df

#Generates ratio features by dividing one feature by another, highlighting the relationships between multiple features
def add_ratios(df, features):
    if len(features) > 1:
        for i in range(len(features)):
            for j in range(i + 1, len(features)):
                df[f"{features[i]}_{features[j]}_ratio"] = df[features[i]] / (df[features[j]] + 1e-9)
    return df


def process_data(data, features):
    data = generate_lag_features(data, features, lags=[1])
    data = add_moving_averages(data, features, windows=[3])
    data = add_diff_features(data, features)
    data = add_ratios(data, features)
    imputer = SimpleImputer(strategy='mean')
    data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns, index=data.index)
    return data_imputed

# Make sure the data is all numeric types
historical_data = combined_data.apply(pd.to_numeric, errors="coerce")
input_data = conditions_2024.apply(pd.to_numeric, errors="coerce")

# Feature generation and filling missing values
features = ["CR2", "CR3", "CR5"]
historical_data_imputed = process_data(historical_data, features)

input_data_imputed = process_data(input_data, features)

        Year  CR2  CR3  CR5  CR2_lag1  CR3_lag1  CR5_lag1   CR2_ma3   CR3_ma3  \
0     2024.0  2.0  3.0  1.0  1.522069  2.104084  4.056324  1.521801  2.103789   
1     2024.0  2.0  3.0  3.0  2.000000  3.000000  1.000000  1.521801  2.103789   
2     2024.0  1.0  2.0  3.0  2.000000  3.000000  3.000000  1.666667  2.666667   
3     2024.0  1.0  1.0  3.0  1.000000  2.000000  3.000000  1.333333  2.000000   
4     2024.0  1.0  1.0  0.0  1.000000  1.000000  3.000000  1.000000  1.333333   
...      ...  ...  ...  ...       ...       ...       ...       ...       ...   
3032  2024.0  1.0  1.0  5.0  1.000000  3.000000  5.000000  1.333333  2.000000   
3033  2024.0  2.0  2.0  3.0  1.000000  1.000000  5.000000  1.333333  2.000000   
3034  2024.0  1.0  3.0  2.0  2.000000  2.000000  3.000000  1.333333  2.000000   
3035  2024.0  2.0  3.0  0.0  1.000000  3.000000  2.000000  1.666667  2.666667   
3036  2024.0  1.0  3.0  5.0  2.000000  3.000000  0.000000  1.333333  3.000000   

       CR5_ma3  CR2_diff  C

In [ ]:
output_cols = {"CR7", "CR11", "CR12", "CR13", "CR21", "CR22", "CR23", "CR54", "CR56"}

# Subtract targets from set and convert the feature to list
common_features_set = set(historical_data_imputed.columns) & set(input_data_imputed.columns)
common_features_list = list(common_features_set - output_cols)

# split dataset
train_data, test_data = train_test_split(historical_data_imputed, test_size=0.2, random_state=42)
# Standardize dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data[common_features_list])
X_test = scaler.transform(test_data[common_features_list])
X_predict = scaler.transform(input_data_imputed[common_features_list])

Choose best model between RandomForestRegressor and XGBRegressor to train each column.

XGBRegressor is based on the gradient boosting framework to capture complex relationship in dataset.

RandomForestRegressor is a regression model that uses an ensemble of decision trees to make predictions to handles large datasets.

However, I find out some columns("CR7", "CR11", "CR12", "CR22", "CR27", "CR56") are hard to capture features in single model, so I combine these two models together to train it.

In [ ]:
def train_and_predict_model(X_train, y_train, X_test, param_grid, model_type='randomforest'):
    if model_type == 'randomforest':
        model = RandomForestRegressor(random_state=42)
    elif model_type == 'xgboost':
        model = XGBRegressor(random_state=42, eval_metric='rmse')

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring="neg_mean_squared_error",
                               cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    return best_model, predictions


def special_processing(X_train, y_train, X_test, X_predict, target):
    # Feature engineering and model optimization strategies
    param_grid_rf_special = {
        "n_estimators": [100, 200],
        "max_depth": [8, 10],
        "min_samples_split": [5, 10],
        "min_samples_leaf": [2, 4]
    }

    best_model_rf, preds_rf = train_and_predict_model(X_train, y_train, X_test, param_grid_rf_special,
                                                      model_type='randomforest')

    param_grid_xgb_special = {
        "learning_rate": [0.01, 0.05],
        "n_estimators": [50, 100],
        "max_depth": [4, 6]
    }

    best_model_xgb, preds_xgb = train_and_predict_model(X_train, y_train, X_test, param_grid_xgb_special,
                                                        model_type='xgboost')

    voting_model = VotingRegressor([('rf', best_model_rf), ('xgb', best_model_xgb)])
    voting_model.fit(X_train, y_train)
    preds_final = voting_model.predict(X_test)


    final_preds = voting_model.predict(X_predict)

    return voting_model, final_preds

In [ ]:
validations = {}
rf_predictions = {}
param_grid_rf = {
    "n_estimators": [50, 100, 150],
    "max_depth": [4, 6, 8],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

param_grid_xgb = {
    "learning_rate": [0.01, 0.05, 0.1],
    "n_estimators": [50, 100, 150],
    "max_depth": [3, 5, 7]
}

special_targets = {"CR7", "CR11", "CR12", "CR22", "CR27", "CR56"}

model_list = [
    ('RandomForest', RandomForestRegressor(random_state=42), param_grid_rf),
    ('XGBoost', XGBRegressor(random_state=42, eval_metric='rmse'), param_grid_xgb)
]

for target in output_cols:
    y_train = train_data[target]
    y_test = test_data[target]

    try:
        if target in special_targets:
            logging.info(f"Applying special processing for target: {target}")
            best_model, best_preds = special_processing(X_train, y_train, X_test, X_predict, target)
        else:
            best_model, best_preds, best_metrics = None, None, None

            for name, model, param_grid in model_list:
                logging.info(f"Training {name} model for target: {target}...")
                best_model_tmp, preds_tmp = train_and_predict_model(X_train, y_train, X_test, param_grid,
                                                                    model_type=name.lower())

                mse = mean_squared_error(y_test, preds_tmp)
                mae = mean_absolute_error(y_test, preds_tmp)
                r2 = r2_score(y_test, preds_tmp)
                accuracy = calculate_accuracy(y_test, preds_tmp)

                metrics = {"MSE": mse, "MAE": mae, "R2": r2, "Accuracy": accuracy}
                logging.info(
                    f"{name} results for {target} - MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}, Accuracy: {accuracy:.4f}")

                if not best_metrics or mse < best_metrics["MSE"]:
                    best_model, best_preds, best_metrics = best_model_tmp, preds_tmp, metrics

            validations[target] = best_metrics

        rf_predictions[target] = best_model.predict(X_predict)

    except Exception as e:
        logging.error(f"Error validating {target} with regression model: {e}")
        rf_predictions[target] = [None] * len(input_data)



print("Validation Results:")
for target, metrics in validations.items():
    if target not in special_targets:
        logging.info(
            f"{target}: MSE: {metrics['MSE']:.4f}, MAE: {metrics['MAE']:.4f}, R2: {metrics['R2']:.4f}, Accuracy: {metrics['Accuracy']:.4f}")
        print(
            f"{target}: MSE: {metrics['MSE']:.4f}, MAE: {metrics['MAE']:.4f}, R2: {metrics['R2']:.4f}, Accuracy: {metrics['Accuracy']:.4f}")

# Store prediction result
final_predictions = {
    i: {target: int(round(rf_predictions.get(target, [None] * len(input_data))[i]))
        for target in output_cols
        if rf_predictions.get(target, [None] * len(input_data))[i] is not None}
    for i in range(len(input_data))
}

# Convert final_predictions to DataFrame
predicted_df = pd.DataFrame.from_dict(final_predictions, orient="index", columns=list(output_cols))

# Ensure the index aligns with the input data
predicted_df.index = input_data.index

desired_order = ['CR7', 'CR11', 'CR12', 'CR13', 'CR21', 'CR22', 'CR23', 'CR54', 'CR56']

# Reorder the columns of predicted_df
predicted_df = predicted_df[desired_order]

# Concatenate inputs_2024 with the predictions
result = pd.concat([inputs_2024.reset_index(drop=True), predicted_df], axis=1)

# Save to CSV file
result.to_csv('Regression_2024_data.csv', index=False)

Validation Results:
CR21: MSE: 0.1496, MAE: 0.2700, R2: 0.0222, Accuracy: 0.6057
CR54: MSE: 0.4811, MAE: 0.3981, R2: 0.0615, Accuracy: 0.5560
CR13: MSE: 0.8109, MAE: 0.3047, R2: 0.0195, Accuracy: 0.6613
CR23: MSE: 0.3489, MAE: 0.1933, R2: 0.0051, Accuracy: 0.7607
